### Step 1: Import Libraries and Download Data

We begin by importing the necessary libraries and downloading historical price data for the 30 stocks in the Dow Jones Industrial Average (DJIA). We'll use `yfinance` to get adjusted closing prices.


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp
import plotly
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


# Set visual style
plt.style.use("seaborn-v0_8")


In [2]:
tickers = ['SPY', 'TLT', 'GLD', 'QQQ', 'EEM']

# Download adjusted close prices
start_date = '2018-01-01'
end_date = '2024-12-31'

# Download adjusted close prices
price_data = yf.download(tickers, start=start_date, end=end_date,multi_level_index=False)['Close']
price_data = price_data.dropna()
price_data.head()


/var/folders/y7/hf6140q96774323j7d756khw0000gn/T/ipykernel_18612/1177253006.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(tickers, start=start_date, end=end_date,multi_level_index=False)['Close']
[*********************100%***********************]  5 of 5 completed


Ticker,EEM,GLD,QQQ,SPY,TLT
Date,,,,,
2018-01-02,40.555485,125.150002,150.779968,238.568771,102.432442
2018-01-03,40.944065,124.820000,152.245026,240.077713,102.922165
2018-01-04,41.146801,125.459999,152.511414,241.089584,102.905823
2018-01-05,41.501587,125.330002,154.043060,242.696228,102.612022
2018-01-08,41.501587,125.309998,154.642395,243.140030,102.546684


### Step 2: Calculate Daily Returns, Mean Returns, and Covariance

We calculate daily log returns from the price data. These are then used to compute annualized mean returns and the covariance matrix, which are inputs to our portfolio simulations.


In [3]:
# Calculate daily log returns
log_returns = np.log(price_data / price_data.shift(1)).dropna()

# Annualized mean returns and covariance matrix
mean_returns = log_returns.mean() * 252
cov_matrix = log_returns.cov() * 252


In [4]:
mean_returns

Ticker
EEM    0.003496
GLD    0.093658
QQQ    0.175772
SPY    0.128434
TLT   -0.025696
dtype: float64

In [5]:
cov_matrix

Ticker,EEM,GLD,QQQ,SPY,TLT
Ticker,,,,,
EEM,0.046051,0.006735,0.038047,0.031983,-0.004518
GLD,0.006735,0.020543,0.004151,0.003083,0.006710
QQQ,0.038047,0.004151,0.058539,0.044114,-0.004580
SPY,0.031983,0.003083,0.044114,0.038181,-0.005606
TLT,-0.004518,0.006710,-0.004580,-0.005606,0.026295


### Step 3: Simulate Portfolios (Shorting Allowed)

We simulate a large number of random portfolios by generating weights using `np.random.randn()`.  
This allows for shorting- some weights may be negative.  
Each portfolio's return, volatility, and Sharpe ratio are computed and stored.


In [6]:
# Set seed for reproducibility
np.random.seed(2)

# Number of portfolios to simulate
n_portfolios = 10_000
n_assets = len(tickers)

# Storage for results
results = {
    'Return': [],
    'Volatility': [],
    'Sharpe': [],
    'Weights': []
}

# Simulate portfolios
for _ in range(n_portfolios):
    # Generate random weights 
    weights = np.random.randn(n_assets)  # Random values from standard normal (can be negative)
    weights /=np.sum(np.abs(weights))      # normalize to sum to 1

    # Calculate expected return
    portfolio_return = np.dot(weights, mean_returns)

    # Calculate portfolio volatility
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Calculate Sharpe Ratio (assuming risk-free rate = 0)
    sharpe_ratio = portfolio_return / portfolio_volatility

    # Store results
    results['Return'].append(portfolio_return)
    results['Volatility'].append(portfolio_volatility)
    results['Sharpe'].append(sharpe_ratio)
    results['Weights'].append(weights)


In [7]:
portfolios_df = pd.DataFrame(results)

### Step 4: Visualize the Raw Efficient Frontier

We convert our simulated results into a DataFrame and use `matplotlib` to visualize the return–risk space.  
Color intensity reflects the Sharpe ratio.


In [8]:
fig = px.scatter(
    portfolios_df,
    x='Volatility',
    y='Return',
    color='Sharpe',
    color_continuous_scale='Viridis',
    title='Raw Efficient Frontier (Shorting Allowed)',
    labels={'Volatility': 'Risk (Volatility)', 'Return': 'Expected Return'},
    width=850,
    height=500
)

fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.show()


In [9]:
# Find the max Sharpe Ratio portfolio
max_sharpe_idx = portfolios_df['Sharpe'].idxmax()
max_point = portfolios_df.iloc[max_sharpe_idx]

# Add to the same Plotly figure
fig.add_scatter(
    x=[max_point['Volatility']],
    y=[max_point['Return']],
    mode='markers',
    marker=dict(size=18, color='red', symbol='star'),
    name='Max Sharpe Portfolio'
)

fig.show()


In [10]:
# Extract optimal weights
optimal_weights = portfolios_df.iloc[max_sharpe_idx]['Weights']
etf_names = tickers  # Assuming `tickers` is your ETF list like ['SPY', 'QQQ', 'GLD', 'TLT', 'EEM']

# Plot optimal weights
fig_weights = px.bar(
    x=etf_names,
    y=optimal_weights,
    labels={'x': 'ETF', 'y': 'Weight'},
    title='Asset Allocation of Max Sharpe Portfolio',
    width=700,
    height=400
)
fig_weights.update_layout(yaxis_tickformat='.0%')
fig_weights.show()


### Step 5: Simulate Long-Only Efficient Frontier (No Shorting)

We now simulate portfolios under a long-only constraint. That means all asset weights must be non-negative and sum to 1 — i.e., no short selling is allowed.

This reflects more realistic scenarios for retail portfolios, ETFs, or long-only funds.


In [11]:
# Storage for long-only results
results_long_only = {
    'Return': [],
    'Volatility': [],
    'Sharpe': [],
    'Weights': []
}

# Simulate long-only portfolios
np.random.seed(42)
for _ in range(n_portfolios):
    weights = np.random.random(n_assets)  # all positive
    weights /= np.sum(weights)            # normalize to sum to 1

    port_return = np.dot(weights, mean_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = port_return / port_volatility

    results_long_only['Return'].append(port_return)
    results_long_only['Volatility'].append(port_volatility)
    results_long_only['Sharpe'].append(sharpe_ratio)
    results_long_only['Weights'].append(weights)


In [12]:
# Convert long-only results to DataFrame
portfolios_long_df = pd.DataFrame(results_long_only)

### Step 6: Visualize Long-Only Efficient Frontier (No Shorting)

Now that we’ve simulated thousands of portfolios with long-only constraints (no shorting), we visualize the resulting efficient frontier.

This shows portfolios composed entirely of positive weights — a realistic setup for mutual funds, ETFs, and individual investors.

We also highlight the portfolio with the highest Sharpe ratio, which represents the best trade-off between risk and return within the long-only universe.


In [27]:
# Find the max Sharpe ratio portfolio
max_sharpe_idx_long = portfolios_long_df['Sharpe'].idxmax()
max_point_long = portfolios_long_df.iloc[max_sharpe_idx_long]

# Plot the efficient frontier (long-only)
fig_long = px.scatter(
    portfolios_long_df,
    x='Volatility',
    y='Return',
    color='Sharpe',
    color_continuous_scale='Viridis',
    title='Efficient Frontier (Long-Only)',
    labels={'Volatility': 'Risk (Volatility)', 'Return': 'Expected Return'},
    width=900,
    height=500
)

# Highlight max Sharpe point
fig_long.add_scatter(
    x=[max_point_long['Volatility']],
    y=[max_point_long['Return']],
    mode='markers',
    marker=dict(size=20, color='red', symbol='star'),
    name='Max Sharpe Portfolio'
)

fig_long.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig_long.show()


### Step 7: Asset Allocation of Long-Only Max Sharpe Portfolio

We now visualize the asset weights for the optimal long-only portfolio — the one with the highest Sharpe ratio.

This helps us understand how capital is distributed across ETFs when the objective is to maximize return relative to risk without shorting.


In [14]:
# Extract optimal weights from long-only simulation
optimal_weights_long = portfolios_long_df.iloc[max_sharpe_idx_long]['Weights']

# Bar plot of asset allocation
fig_weights_long = px.bar(
    x=tickers,
    y=optimal_weights_long,
    labels={'x': 'ETF', 'y': 'Weight'},
    title='Asset Allocation of Long-Only Max Sharpe Portfolio',
    width=700,
    height=400
)
fig_weights_long.update_layout(yaxis_tickformat='.0%')
fig_weights_long.show()


### Step 8: Optimized Portfolios Using a Solver (CVXPY)

We now move from simulation to an exact optimization approach using the `cvxpy` solver.

This step solves for the **maximum Sharpe ratio** portfolio using matrix optimization. The inputs (expected returns and covariance) must be NumPy arrays to avoid compatibility issues with the solver.


In [15]:
# Convert data
mu = mean_returns.to_numpy()
cov = cov_matrix.to_numpy()
n_assets = len(mu)

# Optimization variable
w = cp.Variable(n_assets)

# Define portfolio return and volatility
portfolio_return = mu @ w
portfolio_volatility = cp.quad_form(w, cov)

# Reformulated objective: Maximize return, subject to volatility = 1 (Sharpe logic)
constraints = [
    cp.sum(w) == 1,                      # Full investment
    portfolio_volatility <= 1           # Fix or cap volatility
]

# Define and solve the problem
prob = cp.Problem(cp.Maximize(portfolio_return), constraints)
prob.solve()

# Extract weights
opt_weights_sharpe = w.value

# Display
print("Solver status:", prob.status)
print("Optimal weights (max Sharpe, shorting allowed):")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {opt_weights_sharpe[i]:.4f}")


Solver status: optimal
Optimal weights (max Sharpe, shorting allowed):
SPY: -5.5300
TLT: 4.0754
GLD: 4.4874
QQQ: 0.7076
EEM: -2.7404


### Step 9: Overlay Solver-Based Optimal Portfolio on Efficient Frontier

Now that we've computed the optimal portfolio using a solver (with shorting allowed), we plot it as a red “X” on the efficient frontier of 10,000 simulated portfolios.

This step helps us visually compare:

- The **optimal portfolio** (from mathematical optimization) against the cloud of **randomly simulated portfolios**
- The **Sharpe-maximizing portfolio** lies at the upper-left edge of the frontier, often beyond the scatter if shorting is heavily used
- How simulation-based methods may miss the best point when the number of portfolios is limited (e.g., 10,000)

This visual comparison builds a natural segue into **real-world constraints**, which we’ll start applying next.


In [16]:
# Create DataFrame of simulated portfolios (already exists as portfolios_df)
# portfolios_df['Return'], portfolios_df['Volatility'], portfolios_df['Sharpe']

# Calculate return and volatility of optimal portfolio from solver
opt_return = np.dot(opt_weights_sharpe, mean_returns)
opt_volatility = np.sqrt(np.dot(opt_weights_sharpe.T, np.dot(cov_matrix, opt_weights_sharpe)))

# Plot
fig = px.scatter(
    portfolios_df,
    x='Volatility',
    y='Return',
    color='Sharpe',
    color_continuous_scale='Viridis',
    title='Efficient Frontier with Solver-Based Optimal Portfolio (Shorting Allowed)',
    labels={'Volatility': 'Risk (Volatility)', 'Return': 'Expected Return'},
    width=800,
    height=500
)

# Add red marker for optimal portfolio
fig.add_scatter(
    x=[opt_volatility],
    y=[opt_return],
    mode='markers',
    marker=dict(size=12, color='red', symbol='x'),
    name='Solver Optimal Portfolio'
)

fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.show()


### Step 10: Visualize Optimal Portfolio Weights (Solver-Based, Shorting Allowed)

Now that we've solved for the **Sharpe-optimal portfolio** using a mathematical optimizer (with short selling allowed), we visualize the resulting asset allocations.

Key points to observe:

- **Some weights may be negative**, indicating short positions in certain ETFs
- A few assets may carry large positive or negative allocations-this can be unrealistic in practice without leverage or borrowing constraints
- This highlights why **introducing real-world constraints** is necessary, which we will do in the next step

The bar chart below shows the exact weight assigned to each ETF in the optimized portfolio.


In [17]:
# Create DataFrame for optimal weights (shorting allowed)
opt_weights_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': opt_weights_sharpe
}).sort_values(by='Weight', ascending=False)

# Plot using Plotly
fig = px.bar(round(opt_weights_df,2), x='Ticker', y='Weight',
             title="Optimal Weights (Max Sharpe, Shorting Allowed)",
             labels={'Weight': 'Portfolio Weight'},
             text='Weight')

fig.update_layout(template='plotly_white', yaxis_tickformat=".0%")
fig.show()


### Step 11: Optimize Portfolio (Long-Only Constraint, Solver-Based)

Previously, we allowed **short selling**, which can result in **unrealistically high weights** or even leverage. Now, we'll repeat the optimization, but **restrict weights to be non-negative**, i.e., long-only positions.

Key points:
- This simulates a **realistic retail investor scenario** where only positive holdings are allowed.
- The optimizer will now search for the **maximum Sharpe Ratio** portfolio under this constraint.
- The solution will likely lie within the **efficient frontier** cloud and appear more balanced.

Next, we solve this using `cvxpy`, and then visualize the resulting efficient allocation.


In [18]:
# Define CVXPY variable
w = cp.Variable(n_assets)

# Set a target return (can be tuned)
target_return = mean_returns.mean()

# Objective: minimize portfolio variance
portfolio_variance = cp.quad_form(w, cov_matrix)
objective = cp.Minimize(portfolio_variance)

# Constraints: long-only, full investment, minimum return
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    mean_returns.values @ w >= target_return
]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Extract optimal weights
opt_weights_long_only = w.value


In [19]:
# Print cleanly
print("Long-Only Optimal Weights:")
for ticker, weight in zip(tickers, opt_weights_long_only):
    print(f"{ticker}: {weight:.4f}")

Long-Only Optimal Weights:
SPY: 0.0000
TLT: 0.4206
GLD: 0.0000
QQQ: 0.3285
EEM: 0.2509


### Step 12: Visualize Long-Only Optimal Portfolio on Efficient Frontier

We now overlay the long-only optimal portfolio on the efficient frontier chart. This helps us compare solver-based optimization with the simulation-based frontier (which also included short positions).


In [20]:
# Calculate return and volatility of optimal long-only portfolio
opt_return_long_only = np.dot(opt_weights_long_only, mean_returns)
opt_volatility_long_only = np.sqrt(np.dot(opt_weights_long_only.T, np.dot(cov_matrix, opt_weights_long_only)))

# Plot
fig = px.scatter(
    portfolios_long_df,
    x='Volatility',
    y='Return',
    color='Sharpe',
    color_continuous_scale='Viridis',
    title='Efficient Frontier with Long-Only Optimal Portfolio (Solver-Based)',
    labels={'Volatility': 'Risk (Volatility)', 'Return': 'Expected Return'},
    width=800,
    height=500
)

# Add green marker for long-only optimal portfolio
fig.add_scatter(
    x=[opt_volatility_long_only],
    y=[opt_return_long_only],
    mode='markers',
    marker=dict(size=25, color='red', symbol='star'),
    name='Long-Only Optimal Portfolio',
)

fig.update_layout(coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.show()


### Step 13: Visualize Optimal Portfolio Weights (Solver-Based, Long-Only)

After optimizing the portfolio using **CVXPY with long-only constraints**, we now visualize the resulting asset allocations.

Key observations:

- **All weights are non-negative**, indicating no short-selling  
- The optimizer finds the **maximum Sharpe ratio** portfolio within the feasible long-only space  
- This version is more **realistic and implementable** for most retail and institutional investors  
- Compared to the shorting-allowed case, you'll notice a **more conservative and balanced allocation**

The bar chart below illustrates the weight distribution across ETFs in the long-only optimal portfolio.


In [21]:
# Create DataFrame for optimal weights (long-only)
opt_longonly_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': opt_weights_long_only
}).sort_values(by='Weight', ascending=False)

# Plot using Plotly
fig = px.bar(round(opt_longonly_df,2), x='Ticker', y='Weight',
             title="Optimal Weights (Max Sharpe, Long-Only)",
             labels={'Weight': 'Portfolio Weight'},
             text='Weight')

fig.update_layout(template='plotly_white', yaxis_tickformat=".0%")
fig.show()


### Step 14: Optimize Portfolio with Weight Bounds and Return Floor

We now introduce **realistic portfolio constraints**:

- All weights must lie between **5% and 40%** (to avoid excessive exposure or near-zero allocations)
- Portfolio must be **fully invested** (weights sum to 1)
- Portfolio must achieve a **minimum expected return**

This models a more practical scenario and avoids overly concentrated or negligible allocations.


In [22]:
# Solver-Based Optimization with Allocation Bounds (Long-Only + Floor/Ceiling)

# Convert mean returns to numpy array 
mu = mean_returns.values  # Ensure 1D numpy array
cov = cov_matrix.values   # Also convert covariance matrix to numpy

# Define optimization variable
w = cp.Variable(n_assets)

# Portfolio return and variance
portfolio_return = mu @ w
portfolio_variance = cp.quad_form(w, cov)



# Constraints
target_return = 0.001  # Adjust this floor based on your return expectations
constraints = [
    cp.sum(w) == 1,        # Fully invested
    w >= 0.05,             # At least 5% allocation to each asset
    w <= 0.4,              # At most 40% allocation to each asset
    portfolio_return >= target_return  # Minimum expected return
                  
]

# Objective: Minimize variance under constraints ---
objective = cp.Minimize(portfolio_variance)

# Solve the problem 
problem = cp.Problem(objective, constraints)
problem.solve()

# Extract optimal weights 
opt_weights_min_var_bounded = w.value


In [23]:
# Print cleanly
print("Optimization with Return Constraint Weights:")
for ticker, weight in zip(tickers, opt_weights_min_var_bounded):
    print(f"{ticker}: {weight:.4f}")

Optimization with Return Constraint Weights:
SPY: 0.0500
TLT: 0.3278
GLD: 0.0500
QQQ: 0.2006
EEM: 0.3716


### Step 15: Visualize Long-Only Constrained Portfolio on Efficient Frontier

In this step, we visualize the portfolio optimized using a **minimum variance objective** with a **return floor constraint** and **long-only allocation**.  
This point is plotted over the previously simulated 10,000 random portfolios.
\
**Key observations**:
- This point reflects a realistic portfolio—no shorting and a minimum required return.
- You can now compare how this solution lies relative to the random efficient frontier and other optimized solutions.


In [24]:
# Calculate return and volatility of the long-only constrained optimal portfolio
opt_ret_constrained = np.dot(opt_weights_min_var_bounded, mean_returns)
opt_vol_constrained = np.sqrt(np.dot(opt_weights_min_var_bounded.T, np.dot(cov_matrix, opt_weights_min_var_bounded)))

# Add to efficient frontier scatter plot
fig = px.scatter(
    portfolios_long_df,
    x='Volatility',
    y='Return',
    color='Sharpe',
    color_continuous_scale='Viridis',
    title="Efficient Frontier with Constrained Long-Only Optimal Portfolio",
    labels={'Volatility': 'Risk (Volatility)', 'Return': 'Expected Return'},
    width=800,
    height=500
)

# Add red star marker for constrained portfolio
fig.add_scatter(
    x=[opt_vol_constrained],
    y=[opt_ret_constrained],
    mode='markers',
    marker=dict(size=30, color='red', symbol='star'),
    name='Long-Only Constrained Optimal'
)

fig.update_layout(template='plotly_white', coloraxis_colorbar=dict(title='Sharpe Ratio'))
fig.show()


### Step 16: Bar Plot of Constrained Long-Only Optimal Portfolio Weights

Now that we have solved for the **minimum variance portfolio** with a **required return floor** and **long-only constraint**,  
we plot the resulting weights.

📌 **Key observations**:
- All weights are **positive**, adhering to the long-only constraint.
- The optimizer has allocated higher weights to assets that contribute the least to portfolio volatility while still achieving the target return.


In [25]:
# Create DataFrame for constrained optimal weights
constrained_weights_df = pd.DataFrame({
    'Ticker': tickers,
    'Weight': opt_weights_min_var_bounded
}).sort_values(by='Weight', ascending=False)

# Plot using Plotly
fig = px.bar(
    round(constrained_weights_df,2), x='Ticker', y='Weight',
    title="Optimal Weights (Min Variance with Return Floor, Long-Only)",
    labels={'Weight': 'Portfolio Weight'},
    text='Weight'
)

fig.update_layout(template='plotly_white', yaxis_tickformat=".0%")
fig.show()


In [26]:
# Define helper function
def portfolio_stats(weights, mean_returns, cov_matrix, periods_per_year=252):
    port_return = np.dot(weights, mean_returns)  # no * periods_per_year
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))  # No sqrt(252)
    sharpe = port_return / port_vol
    
    return port_return, port_vol, sharpe

# Store results for each optimized portfolio
results = []

# Portfolio 1: Solver - Max Sharpe (Shorting Allowed)
ret, vol, sharpe = portfolio_stats(opt_weights_sharpe, mean_returns, cov_matrix)
results.append(["Max Sharpe (Shorting Allowed)", ret, vol, sharpe, *opt_weights_sharpe])

# Portfolio 2: Solver - Max Sharpe (Long Only)
ret, vol, sharpe = portfolio_stats(opt_weights_long_only, mean_returns, cov_matrix)
results.append(["Max Sharpe (Long Only)", ret, vol, sharpe, *opt_weights_long_only])

# Portfolio 3: Solver - Min Variance (Bounded, Return Floor)
ret, vol, sharpe = portfolio_stats(opt_weights_min_var_bounded, mean_returns, cov_matrix)
results.append(["Min Variance (Return ≥ 0.1%, 5-40% bounds)", ret, vol, sharpe, *opt_weights_min_var_bounded])

# Create DataFrame
summary_df = pd.DataFrame(results, columns=[
    "Portfolio", "Ann. Return", "Ann. Volatility", "Sharpe Ratio", 
    "SPY", "QQQ", "GLD", "TLT", "EEM"
])

# Round for display
summary_df = summary_df.round(4)

# Show summary
summary_df


,Portfolio,Ann. Return,Ann. Volatility,Sharpe Ratio,SPY,QQQ,GLD,TLT,EEM
0,Max Sharpe (Shorting Allowed),1.3124,1.0000,1.3124,-5.53,4.0754,4.4874,0.7076,-2.7404
1,Max Sharpe (Long Only),0.0751,0.1037,0.7245,0.00,0.4206,0.0000,0.3285,0.2509
2,"Min Variance (Return ≥ 0.1%, 5-40% bounds)",0.0559,0.1028,0.5434,0.05,0.3278,0.0500,0.2006,0.3716
